Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%(my model got 82%).  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

In [57]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk

from IPython.display import display
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

nltk.download('gutenberg')
!python -m spacy download en

/usr/bin/python: No module named spacy


[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/charla/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [39]:
spacy.load('en')

In [40]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])

In [41]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [42]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(Oh, dear, !)",Carroll


In [43]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [44]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


In [59]:
from sklearn.model_selection import train_test_split
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=99)

In [62]:
#Let's see the accuracy scores for all of our models. 
models = []
results = []
names = []
print('\t   Accuracy\n\t score\t  std')

models.append(('NB', GaussianNB()))

models.append(('KNN', KNeighborsClassifier(n_neighbors=3)))

models.append(('LR', LogisticRegression(C=10)))

models.append(('SVM', SVC(kernel='linear',C=10,gamma=0.01)))

models.append(('RF', RandomForestClassifier(max_depth= 14, max_features= 14, n_estimators=5)))

models.append(('XGB',  XGBClassifier(max_depth=2,n_estimators=10,
                                     learning_rate=0.001,subsample=0.2,
                                     colsample_bytree=0.4,colsample_bylevel=0.1)))


for name, model in models:
    kfold = model_selection.KFold(n_splits = 5, random_state = 99)
    cv_results = model_selection.cross_val_score(model, X_train, y_train,
                                                cv = 10, 
                                                 scoring = 'accuracy')
   

    msg = "%s:\t%.4f (%.4f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

	   Accuracy
	 score	  std
NB:	0.8469 (0.0526)
KNN:	0.7185 (0.0347)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework

LR:	0.8507 (0.0822)
SVM:	0.7712 (0.0756)
RF:	0.7604 (0.0496)
XGB:	0.7107 (0.0111)


-----

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

I am trying to see if the model will be good at identifying Hamlet by Shakespeare vs Alice in Wonderland.

In [11]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [12]:
hamlet= gutenberg.raw('shakespeare-hamlet.txt')

In [25]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
hamlet = gutenberg.raw('shakespeare-hamlet.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
hamlet = re.sub(r'Chapter \d+', '', hamlet)
alice = re.sub(r'CHAPTER .*', '', alice)

alice = text_cleaner(alice[:int(len(alice)/10)])
hamlet = text_cleaner(hamlet[:int(len(hamlet)/10)])

In [26]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
hamlet_doc = nlp(hamlet)

In [27]:

# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
hamlet_sents = [[sent, "Shakespeare"] for sent in hamlet_doc.sents]

# Combine sentences
sentences1 = pd.DataFrame(alice_sents+hamlet_sents)
sentences1.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(Oh, dear, !)",Carroll


In [28]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences1, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences1[0]
    df['text_source'] = sentences1[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence1 in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence1
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

In [29]:
# Set up the bags.
alicewords = bag_of_words(alice_doc)
hamletwords = bag_of_words(hamlet_doc)

# Combine bags to create a set of unique words.
common_words1 = set(alicewords + hamletwords)

In [21]:
# Create our data frame with features. This can take a while to run.
word_counts1 = bow_features(sentences1, common_words1)
#word_counts1.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450
Processing row 500
Processing row 550
Processing row 600
Processing row 650
Processing row 700
Processing row 750
Processing row 800
Processing row 850
Processing row 900
Processing row 950
Processing row 1000
Processing row 1050
Processing row 1100
Processing row 1150
Processing row 1200
Processing row 1250
Processing row 1300
Processing row 1350
Processing row 1400
Processing row 1450
Processing row 1500
Processing row 1550
Processing row 1600
Processing row 1650
Processing row 1700
Processing row 1750
Processing row 1800
Processing row 1850
Processing row 1900
Processing row 1950
Processing row 2000
Processing row 2050
Processing row 2100
Processing row 2150
Processing row 2200
Processing row 2250
Processing row 2300
Processing row 2350
Processing row 2400
Processing row 2450
Processing row 2500
Pro

In [49]:
from sklearn.model_selection import train_test_split

In [50]:
y1 = word_counts1['text_source']
X1 = np.array(word_counts1.drop(['text_sentence','text_source'], 1))

'#X1_train1, X1_test1, y1_train1, y1_test1 = train_test_split(X1, \n                                                    y1,\n                                                    test_size=0.4,\n                                                    random_state=99)'

In [59]:
#Let's see the accuracy scores for all of our models. 
models = []
results = []
names = []
print('\t   Accuracy\n\t score\t  std')

models.append(('NB', GaussianNB()))

models.append(('KNN', KNeighborsClassifier(n_neighbors=3)))

models.append(('LR', LogisticRegression(C=10)))

models.append(('SVM', SVC(kernel='linear',C=10,gamma=0.01)))

models.append(('RF', RandomForestClassifier(max_depth= 14, max_features= 14, n_estimators=5)))

models.append(('XGB',  XGBClassifier(max_depth=2,n_estimators=10,
                                     learning_rate=0.001,subsample=0.2,
                                     colsample_bytree=0.4,colsample_bylevel=0.1)))


for name, model in models:
    kfold = model_selection.KFold(n_splits = 5, random_state = 99)
    cv_results = model_selection.cross_val_score(model, X1, y1,
                                                cv = 10, 
                                                 scoring = 'accuracy')
   
    #ypred=model.predict(X1_test)
    msg = "%s:\t%.4f (%.4f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

	   Accuracy
	 score	  std
NB:	0.9389 (0.0244)
KNN:	0.9558 (0.0215)
LR:	0.9828 (0.0049)
SVM:	0.9816 (0.0044)
RF:	0.9650 (0.0039)
XGB:	0.9617 (0.0009)


Linear Regression correctly predicted 98.28% of the time.

In [58]:
'''
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, 
                                                    y1,
                                                    test_size=0.4,
                                                    random_state=99)
'''
lr= LogisticRegression()
train = lr.fit(X1_train, y1_train)
ypred=lr.predict(X1_test)
pd.crosstab(y1_test, ypred)


print('Training set score:', lr.score(X1_train, y1_train))
print('\nTest set score:', lr.score(X1_test, y1_test))



Training set score: 0.9881305637982196

Test set score: 0.9740548554484804


The model is good at predicting Carroll from Shakespeare. It is actually able to differentiate multiple works by Shakespeare from Alice in Wonderland. 